In [1]:
from bs4 import BeautifulSoup  # Importación correcta de BeautifulSoup
import requests  # Importación correcta de requests
import json  # Importación correcta de json
import re

#----------------------------VARIABLES GLOBALES--------------------------------
URL_COLMO="https://www.colmo.com.ar/productos"
URL_DALA="https://dalaimpresion.com/productos"
URL_ADIDAS=""
URL_MONTAGNE="https://www.montagne.com.ar"
URL_KABRAKUERVO="https://www.kabrakuervo.com/search/?q="

PARAMETRO={
    'page':1,
    'limit':12
}
USER_AGENT={
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0'
}
USER_AGENT_MONTAGNE={
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36 Edg/134.0.0.0'
}

#---------------------------FUNCIONES MONTAGNE--------------------------------------
def obtenerLinksMontagne(htmlInicio):
    filtro=htmlInicio.find(id="header-products").find_all("li")
    lista_ul=[]
    for linea in filtro:
        busquedaLinea=linea.find_all(class_="level-3")
        if busquedaLinea!=[] and len(busquedaLinea)!=1:
            lista_ul.append(busquedaLinea)

    lista_ul=[tupla for sublista in lista_ul for tupla in sublista]
    links = []
    for linea in lista_ul:
        a = linea.find('a', href=True)
        links.append(a['href'])
    links=list(set(links))
    return links


def obtenerIds(html):
    listaIDs=[]
    lista=html.find_all(class_="product-fav-btn")
    for item in lista:
        listaIDs.append(item["data-idproduct"])
    return listaIDs

def ObtenerPrecioFinal(precioA, precioB):
    precioA = -1 if precioA==None else precioA.get_text() ##-1 SIGNIFICA EN TIENDAS y que no hay precio
    precioB = -1 if precioB==None else precioB.get_text()
    if precioA == precioB ==-1:
        return -1
    if precioA ==-1:
        return ''.join(re.findall(r'\d+', precioB))
    else: return int(''.join(re.findall(r'\d+', precioA)))

def hola(texto):

    return texto

#------------------------------------FUNCIONES DALA----------------------------------------
def esOfertaDala(descripcion):
    return "remate" in descripcion.lower()


#-----------------------------FUNCIONES KABRAKUERVO--------------------------
def esOfertaKabraKuervo(linea):
    if linea.find(class_="item-price-container mb-1").find(class_="js-compare-price-display price-compare", style="display:inline-block;")!= None: #A
        return True
    return False

#-----------------------------FUNCIONES COLMO-----------------------------
def esOfertaColmo(linea):
        if(linea)!=None: #podria xtraer para soloesOfertaColmo()
            return True
        return False


#------------- FUNCIONES GENERALES---------------------------------
def obtenerPrecioEntero(precioTexto):
    return ''.join(re.findall(r'\d+', precioTexto))


def obtenerHtml(url,user_agent,parametros=None):
    respuesta=requests.get(url,headers=user_agent,params=parametros) #request con parametro de $pagina actualizado
    return BeautifulSoup(respuesta.text,'html') #parsear html
    



#---------------------FUNCIONES PARA OBTENER LISTAS PARCIALES DE LOS PRODUCTOS--------------------
def obtenerListaDala(pagina):
    listaProductos=[]
    PARAMETRO['page']=pagina
    htmlDala=obtenerHtml(URL_DALA,USER_AGENT,PARAMETRO) 
    busqueda=htmlDala.find_all(class_='js-item-product item m-top-none-xs') 
    
    if busqueda==[]:return listaProductos
    for linea in busqueda:
        idProducto=linea.get("data-product-id")
        containerProductos=linea.find_all(class_="js-product-container js-quickshop-container js-quickshop-has-variants")
        for cadena in containerProductos: 
            data_variants=json.loads(cadena.get("data-variants"))
            precio=obtenerPrecioEntero(data_variants[0]["price_short"])
            descripcion=(cadena.find('a').get("title"))
            listaProductos.append((idProducto,descripcion,precio,esOfertaDala(descripcion))) 
    
    return listaProductos


def obtenerListaKabraKuervo(page):
   listaProductos=[]
   PARAMETRO["page"]=page
   htmlKabraKuervo=obtenerHtml(URL_KABRAKUERVO,USER_AGENT,PARAMETRO)
   busqueda=htmlKabraKuervo.find_all(class_="js-item-product col-md-4 item item-product")
   if busqueda==[] :return listaProductos
   for linea in busqueda:
      data_variants=json.loads(linea.find(class_="js-product-container js-quickshop-container item-container").get("data-variants"))
      precio=obtenerPrecioEntero(data_variants[0]["price_short"])
      idProducto=data_variants[0]["product_id"]
      nombre=(linea.find('a').get("title"))
      tupla=tuple((idProducto,nombre,precio,esOfertaKabraKuervo(linea))) 
      listaProductos.append(tupla)
   return listaProductos


def obtenerListaColmo(page):
    PARAMETRO['page']=page
    listaProductos=[]
    htmlColmo=obtenerHtml(URL_COLMO,USER_AGENT,PARAMETRO) 
    busqueda=htmlColmo.find_all(class_='js-item-product col-6 col-md-3 item item-product') 
    if busqueda==[]: return listaProductos 
    for producto in busqueda:
        oferta=esOfertaColmo(producto.find(class_='js-offer-label label label-accent'))
        tupla=(producto.get('data-product-id'),producto.find('a').get('title'),obtenerPrecioEntero(json.loads(producto.find('script').string)['offers']['price']),oferta)    
        listaProductos.append(tupla)
    return listaProductos



def obtenerListaMontagne(link):
    listaProductos=[]
    
    busqueda=requests.get(link,headers=USER_AGENT_MONTAGNE)
    html=BeautifulSoup(busqueda.text,"html")
    lista=html.find_all(class_="product-list-info-container")
    ListaIds=obtenerIds(html)
    iterador=0
    for linea in lista:
        precioRegular=linea.find(class_="price-regular price-old")
        precioOferta=linea.find(class_="price-offer")
        oferta= precioOferta!=None
        precio=ObtenerPrecioFinal(precioRegular,precioOferta)
        descripcion=linea.find(class_="product-list-info-title").find('div').get_text() + " " + linea.find(class_="product-list-info-title").find('h3').get_text()
        listaProductos.append((ListaIds[iterador],descripcion,precio,oferta))
        iterador=iterador+1
    return listaProductos



##------------------------------OBTENCION DE PRODUCTOS --------------------------
def obtenerProductosDala():
    listaProductos=[]
    pagina=1
    productosPagina=obtenerListaDala(pagina)
    while productosPagina!=[]:
        listaProductos.append(productosPagina)
        pagina=pagina+1        
        productosPagina=obtenerListaDala(pagina)
    listaProductos=[tupla for sublista in listaProductos for tupla in sublista]
    data = [{"id": producto[0], "descripcion": producto[1], "precio": producto[2], "oferta": producto[3]} for producto in listaProductos]
    return data


def obtenerProductosKabraKuervo():
   listaProductos=[]
   pagina=1
   productosPagina=obtenerListaKabraKuervo(pagina)
   while productosPagina!=[]:
      listaProductos.append(productosPagina)
      pagina=pagina+1
      productosPagina=obtenerListaKabraKuervo(pagina)
   listaProductos=[tupla for sublista in listaProductos for tupla in sublista]
   data = [{"id": producto[0], "descripcion": producto[1], "precio": producto[2], "oferta": producto[3]} for producto in listaProductos]
    
   return data


def obtenerProductosColmo():
    listaProductos=[]
    pagina=1
    
    productosPagina=obtenerListaColmo(pagina)
    while productosPagina!=[]:
        listaProductos.append(productosPagina)
        pagina=pagina+1
        productosPagina=obtenerListaColmo(pagina)
    listaProductos=[tupla for sublista in listaProductos for tupla in sublista]
    data = [{"id": producto[0], "descripcion": producto[1], "precio": producto[2], "oferta": producto[3]} for producto in listaProductos]
    
    return data




def obtenerProductosMontagne():
    html=obtenerHtml(URL_MONTAGNE,USER_AGENT)
    links=obtenerLinksMontagne(html)
    listaProductos=[]
    for direccion in links:
        direccionFinal=URL_MONTAGNE+direccion
        listaProductos.append(obtenerListaMontagne(direccionFinal))
    
    listaProductos=[tupla for sublista in listaProductos for tupla in sublista]
    data = [{"id": producto[0], "descripcion": producto[1], "precio": producto[2], "oferta": producto[3]} for producto in listaProductos]
    
    return data
